In [71]:
import numpy as np
import pandas as pd
import seaborn as sns
import copy
import warnings

warnings.filterwarnings('ignore')

In [72]:
scores_df = pd.read_csv('C:/Users/Gabriel/Documents/GitHub/MIE368_PROJECT/team_proj/data/season_data.csv')
playoff_scores_df = pd.read_csv('C:/Users/Gabriel/Documents/GitHub/MIE368_PROJECT/team_proj/data/playoff_data.csv')
real_elo = pd.read_csv('C:/Users/Gabriel/Documents/GitHub/MIE368_PROJECT/team_proj/data/nhl_elo.csv')

In [73]:
scores_df.head()

,Date,Visitor,G_Vis,Home,G_Home,OT Ind,Season
0,2000-10-04,Colorado Avalanche,2.0,Dallas Stars,2.0,1,2001
1,2000-10-05,Ottawa Senators,4.0,Boston Bruins,4.0,1,2001
2,2000-10-05,Chicago Blackhawks,2.0,Buffalo Sabres,4.0,0,2001
3,2000-10-05,Detroit Red Wings,4.0,Calgary Flames,3.0,0,2001
4,2000-10-05,Vancouver Canucks,3.0,Philadelphia Flyers,6.0,0,2001


## Game Result Elo Adjustment

In [74]:
first_yr = scores_df[scores_df['Season'] == 2001]
Teams = first_yr['Home'].unique().tolist()

team_changes = {
    'Atlanta Thrashers': 'Winnipeg Jets',
    'Phoenix Coyotes': 'Arizona Coyotes',
    'Mighty Ducks of Anaheim': 'Anaheim Ducks'
    # Add more mappings as needed
}

K = 6

In [75]:
def calculate_prob_winning(home_team, away_team, elo_ratings):
    
    Elo_diff_home = elo_ratings[home_team]-elo_ratings[away_team] + 26  #add extra for home-ice advantage   
    #Elo_diff_away = elo_ratings[away_team] - elo_ratings[home_team]
    prob_win_home = 1/(10**(-1*Elo_diff_home/400)+1)
    #prob_win_away = 1/(10**(-1*Elo_diff_away/400)+1)
    prob_win_away = 1 - prob_win_home
    
    return prob_win_home, prob_win_away, Elo_diff_home

In [76]:
def margin_of_victory(home_goals, away_goals):
    
    mov = abs(home_goals - away_goals)
    mov_mult = 0.6686*np.log(mov)+0.8048
    
    return mov_mult

In [77]:
# Function to update ELO ratings after a game -- Could add input for elo_ratings dictionary 
def update_game_elo(home_team, away_team, home_goals, away_goals, season, df, idx, elo_ratings):
    
    # Map old team names to new ones
    home_team = team_changes.get(home_team, home_team)
    away_team = team_changes.get(away_team, away_team)
    
    #print('Home team is ' + str(home_team))
    #print('Away team is ' + str(away_team))

    if home_team not in elo_ratings:
        # New teams added past a certain year start with a different ELO rating
        elo_ratings[home_team] = 1500
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1500

     # Calculate the probability of winning the game for each team
    home_prob, away_prob, Elo_diff_home = calculate_prob_winning(home_team, away_team, elo_ratings)
    
    # Update DataFrame values
    df.at[idx, 'EloDiffHome'] = Elo_diff_home
    df.at[idx, 'EloDiffAway'] = -1 * Elo_diff_home
    df.at[idx, 'WinProbHome'] = home_prob
    df.at[idx, 'WinProbVis'] = away_prob
    
     # Update Pregame Favorite Multiplier
    if home_goals > away_goals:
        home_win = 1
        #away_win = 0
        winner_elo_diff = Elo_diff_home
    elif home_goals < away_goals:
        home_win = 0
        #away_win = 1
        winner_elo_diff = -1*Elo_diff_home
    else:
        return df, elo_ratings
    
    pre_g_fav_h = home_win - home_prob  
    #pre_g_fav_a = away_win - away_prob  
    
    auto_corr = 2.05/(winner_elo_diff*0.001 + 2.05)
    
     # Adjust ELO shift for margin of victory
    mov_multiplier = auto_corr * margin_of_victory(home_goals, away_goals)
    #print('home goals:' + str(home_goals))
    #print('away goals:' + str(away_goals))
    #print('mult is: ' + str(mov_multiplier))
    
     # Calculate ELO shift based on game result
    elo_shift_h = K * mov_multiplier * pre_g_fav_h 
    #elo_shift_a = K * mov_multiplier * pre_g_fav_a
    #print('elo shift is: ' + str(elo_shift_h))
    
    elo_ratings[home_team] += elo_shift_h
    elo_ratings[away_team] -= elo_shift_h

    return df, elo_ratings

## End of Season Elo Adjustments

In [78]:
def elo_adjuster(df):
    
    # Initialize variables to store the final Elo ratings for the season
    final_elo_ratings = {}
    elo_ratings = {}
    season_weight = 0.8
    average_weight = 0.2
    current_season = None

    # Iterate over seasons and games
    for idx, row in df.iterrows():
        if current_season is None:
            current_season = row['Season']
    
        # Check if the season in the current row is different from the current season
        if row['Season'] != current_season:
            # Save the previous season's Elo ratings and calculate the starting Elo for the new season
            final_elo_ratings[current_season] = dict(elo_ratings)
            elo_ratings = {}  # Reset Elo ratings for the new season

            for team in final_elo_ratings[current_season]:
                prev_season_elo = final_elo_ratings[current_season][team]
                league_avg_elo = sum(final_elo_ratings[current_season].values()) / len(final_elo_ratings[current_season])
                starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
                elo_ratings[team] = round(starting_elo, 2)

            current_season = row['Season']
            
        # Update Elo ratings for the game
        df, elo_ratings = update_game_elo(row['Home'], row['Visitor'], row['G_Home'], row['G_Vis'], row['Season'], df, idx, elo_ratings)

    # Calculate the final Elo ratings for the last season
    final_elo_ratings[current_season] = dict(elo_ratings)

    return df, elo_ratings, final_elo_ratings

In [79]:
seasons, elo, final_season_elos = elo_adjuster(scores_df)

In [80]:
elo

{'Dallas Stars': 1550.940150407058,
 'Colorado Avalanche': 1584.5444245615597,
 'Boston Bruins': 1646.0070614177143,
 'Ottawa Senators': 1476.0714294103982,
 'Buffalo Sabres': 1483.3405044471829,
 'Chicago Blackhawks': 1390.3846749520826,
 'Calgary Flames': 1517.6920572246158,
 'Detroit Red Wings': 1438.9591928296538,
 'Philadelphia Flyers': 1417.745218787118,
 'Vancouver Canucks': 1484.9400838907918,
 'Arizona Coyotes': 1401.4428712249032,
 'St. Louis Blues': 1486.1607904424352,
 'Edmonton Oilers': 1582.4085250119847,
 'Florida Panthers': 1541.7356119720528,
 'Anaheim Ducks': 1365.7461410120907,
 'Minnesota Wild': 1547.1258135936355,
 'New Jersey Devils': 1529.031110424827,
 'Montreal Canadiens': 1405.9111077584168,
 'Pittsburgh Penguins': 1507.3256629722844,
 'Nashville Predators': 1506.5670113535894,
 'San Jose Sharks': 1382.9101359293545,
 'Tampa Bay Lightning': 1542.7115664597802,
 'New York Islanders': 1513.7599713609845,
 'Washington Capitals': 1477.297610220473,
 'Los Angeles K

### Testing Bounds of Elo

In [11]:
for i in final_season_elos:
    print(f'Max of year {i} is {max(final_season_elos[i].values())}')
    print(f'Min of year {i} is {min(final_season_elos[i].values())}')

Max of year 2001 is 1592.1355800566328
Min of year 2001 is 1407.17797613946
Max of year 2002 is 1594.5595423047896
Min of year 2002 is 1375.969112961436
Max of year 2003 is 1604.4834324889157
Min of year 2003 is 1403.313756619737
Max of year 2004 is 1600.703277602256
Min of year 2004 is 1381.994928786802
Max of year 2006 is 1621.8315238665518
Min of year 2006 is 1385.2106980872195
Max of year 2007 is 1585.809806922114
Min of year 2007 is 1410.9965387341117
Max of year 2008 is 1592.415920121985
Min of year 2008 is 1439.730361842258
Max of year 2009 is 1573.4170823439883
Min of year 2009 is 1398.4869054830074
Max of year 2010 is 1584.8075179050084
Min of year 2010 is 1423.9071652779273
Max of year 2011 is 1591.4007466678427
Min of year 2011 is 1405.5966526380455
Max of year 2012 is 1577.0127514839012
Min of year 2012 is 1431.492209051298
Max of year 2013 is 1591.2735552272406
Min of year 2013 is 1419.8150830163438
Max of year 2014 is 1588.4750973383004
Min of year 2014 is 1394.2163417588

In [16]:
season_weight = 0.8
average_weight = 0.2
max_min = {}
min_ls = []
max_ls = []

for i in final_season_elos:
    start_elo = {}
    for team in final_season_elos[i]:
        prev_season_elo = final_season_elos[i][team]
        league_avg_elo = sum(final_season_elos[i].values()) / len(final_season_elos[i])
        starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
        start_elo[team] = round(starting_elo, 2)
    max_val = max(start_elo.values())
    min_val = min(start_elo.values())
    max_ls.append(max_val)
    min_ls.append(min_val)
    max_min[str(i)+' max'] = max_val
    max_min[str(i)+' min'] = min_val

In [20]:
print(f'The absolute max is {max(max_ls)}')
print(f'The absolute min is {min(min_ls)}')
print(f'The average max is {sum(max_ls) / len(max_ls)}')
print(f'The average min is {sum(min_ls) / len(min_ls)}')

The absolute max is 1616.81
The absolute min is 1388.25
The average max is 1574.9254545454548
The average min is 1419.6604545454543


In [17]:
max_min

{'2001 max': 1573.71,
 '2001 min': 1425.74,
 '2002 max': 1575.65,
 '2002 min': 1400.78,
 '2003 max': 1583.59,
 '2003 min': 1422.65,
 '2004 max': 1580.56,
 '2004 min': 1405.6,
 '2006 max': 1597.47,
 '2006 min': 1408.17,
 '2007 max': 1568.65,
 '2007 min': 1428.8,
 '2008 max': 1573.93,
 '2008 min': 1451.78,
 '2009 max': 1558.73,
 '2009 min': 1418.79,
 '2010 max': 1567.85,
 '2010 min': 1439.13,
 '2011 max': 1573.12,
 '2011 min': 1424.48,
 '2012 max': 1561.61,
 '2012 min': 1445.19,
 '2013 max': 1573.02,
 '2013 min': 1435.85,
 '2014 max': 1570.78,
 '2014 min': 1415.37,
 '2015 max': 1563.92,
 '2015 min': 1389.55,
 '2016 max': 1558.31,
 '2016 min': 1435.69,
 '2017 max': 1578.44,
 '2017 min': 1408.81,
 '2018 max': 1556.13,
 '2018 min': 1422.12,
 '2019 max': 1598.84,
 '2019 min': 1428.51,
 '2020 max': 1574.84,
 '2020 min': 1388.25,
 '2021 max': 1565.67,
 '2021 min': 1425.16,
 '2022 max': 1576.73,
 '2022 min': 1419.51,
 '2023 max': 1616.81,
 '2023 min': 1392.6}

## Building Elo Forecaster Function

In [81]:
#np.random.seed(0)
def forecast_game_elo(home_team, away_team, season, df, idx, elo_ratings, simulation_wins):

    # Map old team names to new ones
    home_team = team_changes.get(home_team, home_team)
    away_team = team_changes.get(away_team, away_team)

    if home_team not in elo_ratings:
        elo_ratings[home_team] = 1500
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1500

     # Calculate the probability of winning the game for each team
    home_prob, away_prob, Elo_diff_home = calculate_prob_winning(home_team, away_team, elo_ratings)

    # Update DataFrame values
    df.at[idx, 'EloDiffHome'] = Elo_diff_home
    df.at[idx, 'EloDiffAway'] = -1 * Elo_diff_home
    df.at[idx, 'WinProbHome'] = home_prob
    df.at[idx, 'WinProbVis'] = away_prob
    
    home_win = np.random.choice([0, 1], p=[away_prob, home_prob])

    # Calculate base scores for the home and away team
    score_home = 2.845905 + 0.003584*Elo_diff_home
    score_away = 2.845905 + 0.003584*(-1)*Elo_diff_home

    conditions_satisfied = 0 

    while conditions_satisfied == 0:
        
        # poisson distribution to shift scores
        poisson_score_home = np.random.poisson(max(0, score_home))
        poisson_score_away = np.random.poisson(max(0, score_away))

        home_goal_diff = poisson_score_home - poisson_score_away
            
        # does it satsify conditions?
        if home_win == 1:
            if home_goal_diff > 0:
                conditions_satisfied = 1
        
        if home_win == 0:
            if home_goal_diff < 0: 
                conditions_satisfied = 1

    #------------------------------------------------------------------------#
    OT_choice = 0
    
    if abs(home_goal_diff) == 1:
        # Decide if the game goes to OT
        OT_choice = np.random.choice(a= [1, 0], p= [0.505, 0.495])
            
    if home_win == 1:
        winner_elo_diff = Elo_diff_home
        simulation_wins[home_team] += 2

        if OT_choice == 1:
            simulation_wins[away_team] += 1
        
    if home_win == 0:
        winner_elo_diff = -1*Elo_diff_home
        simulation_wins[away_team] += 2

        if OT_choice == 1:
            simulation_wins[home_team] += 1

    pre_g_fav_h = home_win - home_prob  
    auto_corr = 2.05/(winner_elo_diff*0.001 + 2.05)
    
    # Adjust ELO shift for margin of victory
    mov_multiplier = auto_corr * margin_of_victory(poisson_score_home, poisson_score_away)

    # Calculate ELO shift based on game result
    elo_shift_h = K * mov_multiplier * pre_g_fav_h 
  
    elo_ratings[home_team] += elo_shift_h
    elo_ratings[away_team] -= elo_shift_h

    df.at[idx, 'G_Home'] = poisson_score_home
    df.at[idx, 'G_Vis'] = poisson_score_away
    df.at[idx, 'OT Ind'] = OT_choice

    #--------------------------------------------------------------------------#

    return df, elo_ratings, simulation_wins

In [82]:
# Iterate over seasons and games
def season_forecaster(df, starting_elo):
        
    # Create a dictionary to track point totals
    simulation_pts = {team: 0 for team in df['Home'].unique()}

    for idx, row in df.iterrows():
        df, starting_elo, simulation_pts = forecast_game_elo(row['Home'], row['Visitor'], row['Season'], df, idx, starting_elo, simulation_pts)  
        
    return df, starting_elo, simulation_pts

In [83]:
def playoff_tracker(simulation_pts, playoffs_made):

    conference_mapping = {
        'East': ['Florida Panthers', 'Toronto Maple Leafs', 'Tampa Bay Lightning', 'Boston Bruins', 'Buffalo Sabres', 'Detroit Red Wings',
                 'Ottawa Senators', 'Montreal Canadiens', 'Carolina Hurricanes', 'New York Rangers', 'Pittsburgh Penguins', 
                 'Washington Capitals', 'New York Islanders', 'Columbus Blue Jackets', 'New Jersey Devils', 'Philadelphia Flyers'],
        'West': ['Colorado Avalanche', 'Minnesota Wild', 'St. Louis Blues', 'Dallas Stars', 'Nashville Predators', 'Winnipeg Jets',
                 'Chicago Blackhawks', 'Arizona Coyotes', 'Calgary Flames', 'Edmonton Oilers', 'Los Angeles Kings', 'Vegas Golden Knights',
                 'Vancouver Canucks', 'San Jose Sharks', 'Anaheim Ducks', 'Seattle Kraken']
    }
    
    # Separate teams into East and West lists
    for conference, teams in conference_mapping.items():
        valid_teams = [i for i in teams if i in simulation_pts]
        sorted_teams = sorted(valid_teams, key=lambda x: simulation_pts[x], reverse=True)

        for team in sorted_teams[:8]:
            playoffs_made[team] += 1

    return playoffs_made

In [84]:
np.random.seed(0)

df2 = copy.deepcopy(scores_df[scores_df['Season'] == 2023])
season_2022 = df2.copy() 
num_simulations = 10000
season_weight = 0.8
average_weight = 0.2

# Create a dictionary to track overall standings
standings = {team: np.zeros(num_simulations) for team in df2['Home'].unique()}

# Create a dictionary to track playoff appearances
playoffs_counter = {team: 0 for team in df2['Home'].unique()}

for simulation in range(num_simulations):
    elo_end = copy.deepcopy(final_season_elos[2021])  # Creating a deep copy of final_season_elos[2021]

    elo_ratings = {}  # Reset Elo ratings for the new season

    for team in elo_end:
        prev_season_elo = elo_end[team]
        league_avg_elo = sum(elo_end.values()) / len(elo_end)
        starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
        elo_ratings[team] = round(starting_elo, 2)

    season_2022, elo_ratings, simulation_pts = season_forecaster(season_2022, elo_ratings)
    playoffs_counter = playoff_tracker(simulation_pts, playoffs_counter)

    for team, pts in simulation_pts.items():
        standings[team][simulation] = pts


for team, pts in standings.items():
    #print(pts)
    mean_pts = np.mean(pts)
    std_deviation = np.std(pts)
    print(f"{team}: Mean point total = {mean_pts}, Standard Deviation = {std_deviation}")

Nashville Predators: Mean point total = 93.3676, Standard Deviation = 12.3481525031075
San Jose Sharks: Mean point total = 78.1848, Standard Deviation = 12.298245767588156
Los Angeles Kings: Mean point total = 79.4855, Standard Deviation = 12.344706952779397
New York Rangers: Mean point total = 91.6652, Standard Deviation = 12.350842439283241
Anaheim Ducks: Mean point total = 75.309, Standard Deviation = 12.193855788879905
Carolina Hurricanes: Mean point total = 100.3925, Standard Deviation = 12.051839849168259
Colorado Avalanche: Mean point total = 103.3856, Standard Deviation = 11.979520551340944
Edmonton Oilers: Mean point total = 94.9998, Standard Deviation = 12.282906820455816
Montreal Canadiens: Mean point total = 83.3457, Standard Deviation = 12.314137871162561
Washington Capitals: Mean point total = 100.4537, Standard Deviation = 11.956598860461948
Buffalo Sabres: Mean point total = 73.9881, Standard Deviation = 12.270906991335236
Calgary Flames: Mean point total = 88.8539, Sta

### Finding Runtime Issues

In [19]:
import cProfile

def main():
    season_2022 = copy.deepcopy(scores_df[scores_df['Season'] == 2022])
    num_simulations = 500
    season_weight = 0.7
    average_weight = 0.3

    # Create a dictionary to track overall standings
    standings = {team: np.zeros(num_simulations) for team in df2['Home'].unique()}

    # Create a dictionary to track playoff appearances
    playoffs_counter = {team: 0 for team in df2['Home'].unique()}

    for simulation in range(num_simulations):
        elo_end = copy.deepcopy(final_season_elos[2021])  # Creating a deep copy of final_season_elos[2021]

        elo_ratings = {}  # Reset Elo ratings for the new season

        for team in elo_end:
            prev_season_elo = elo_end[team]
            league_avg_elo = sum(elo_end.values()) / len(elo_end)
            starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
            elo_ratings[team] = round(starting_elo, 2)

        season_2022, elo_ratings, simulation_pts = season_forecaster(season_2022, elo_ratings)
        playoffs_counter = playoff_tracker(simulation_pts, playoffs_counter)

        for team, pts in simulation_pts.items():
            standings[team][simulation] = pts


    for team, pts in standings.items():
        #print(pts)
        mean_pts = np.mean(pts)
        std_deviation = np.std(pts)
        print(f"{team}: Mean point total = {mean_pts}, Standard Deviation = {std_deviation}")

if __name__ == "__main__":
    # Run the profiler
    cProfile.run('main()', sort='cumulative')

         2082 function calls (1890 primitive calls) in 0.005 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.005    0.005 {built-in method builtins.exec}
        1    0.000    0.000    0.005    0.005 <string>:1(<module>)
        1    0.000    0.000    0.005    0.005 3134879366.py:3(main)
        1    0.000    0.000    0.002    0.002 4235467807.py:2(season_forecaster)
        1    0.000    0.000    0.001    0.001 indexing.py:2481(__getitem__)
        1    0.000    0.000    0.001    0.001 indexing.py:2431(__getitem__)
        3    0.000    0.000    0.001    0.000 base.py:3762(get_loc)
        1    0.000    0.000    0.001    0.001 frame.py:3982(_get_value)
        5    0.000    0.000    0.001    0.000 frame.py:3853(__getitem__)
     64/2    0.000    0.000    0.001    0.000 copy.py:128(deepcopy)
        3    0.001    0.000    0.001    0.000 {method 'get_loc' of 'pandas._libs.index.IndexEngine'

KeyError: 0

### Forecast Results

In [85]:
len(season_2022[season_2022['OT Ind'] == 1])/len(season_2022)

0.17530487804878048

In [86]:
mean_pts = pd.DataFrame(columns = ['PTS']).rename_axis('Team')

for team, pts in standings.items():
    mean_pts.loc[team] = [np.mean(pts)]



In [87]:
mean_pts.sort_values(by='PTS', ascending=False)

,PTS
Team,
Vegas Golden Knights,104.6735
Colorado Avalanche,103.3856
Boston Bruins,101.9320
Tampa Bay Lightning,101.8273
Pittsburgh Penguins,101.1079
Washington Capitals,100.4537
Carolina Hurricanes,100.3925
Florida Panthers,98.1412
Toronto Maple Leafs,97.7573


In [60]:
print("Elo ratings at the end of the simulation")

elo_df = pd.DataFrame(columns = ['Elo']).rename_axis('Team')

for team, elo in elo_ratings.items():
    elo_df.loc[team] = [elo]

elo_df.sort_values(by='Elo', ascending=False)

Elo ratings at the end of the simulation


,Elo
Team,
St. Louis Blues,1587.764744
Vegas Golden Knights,1580.006114
Florida Panthers,1569.468401
Toronto Maple Leafs,1568.412368
Colorado Avalanche,1557.977873
Minnesota Wild,1557.702944
Pittsburgh Penguins,1547.118887
New York Islanders,1544.203893
New York Rangers,1544.172651


In [61]:
final_season_elos[2022]

{'Dallas Stars': 1503.2303905512265,
 'Colorado Avalanche': 1595.910441864351,
 'Boston Bruins': 1564.5113773543303,
 'Ottawa Senators': 1447.7714727082273,
 'Buffalo Sabres': 1438.1486602939285,
 'Chicago Blackhawks': 1417.6440396028033,
 'Calgary Flames': 1562.634961904245,
 'Detroit Red Wings': 1409.9220590200669,
 'Philadelphia Flyers': 1404.0107526135457,
 'Vancouver Canucks': 1502.767612472211,
 'Arizona Coyotes': 1406.5800019869648,
 'St. Louis Blues': 1563.6611183837383,
 'Edmonton Oilers': 1552.6733530203833,
 'Florida Panthers': 1591.236730017949,
 'Anaheim Ducks': 1419.6543663507791,
 'Minnesota Wild': 1569.853125584181,
 'New Jersey Devils': 1410.4647319496478,
 'Montreal Canadiens': 1399.389815598266,
 'Pittsburgh Penguins': 1549.7300127663448,
 'Nashville Predators': 1518.0108034350656,
 'San Jose Sharks': 1421.0856741433977,
 'Tampa Bay Lightning': 1571.8677746004355,
 'New York Islanders': 1502.759514458031,
 'Washington Capitals': 1536.7475211405108,
 'Los Angeles King

In [62]:
print("Elo ratings at the end of the real prior season")

elo_df = pd.DataFrame(columns = ['Elo']).rename_axis('Team')

for team, elo in final_season_elos[2022].items():
    elo_df.loc[team] = [elo]

elo_df.sort_values(by='Elo', ascending=False)

Elo ratings at the end of the real prior season


,Elo
Team,
Colorado Avalanche,1595.910442
Florida Panthers,1591.236730
Carolina Hurricanes,1585.072572
Toronto Maple Leafs,1575.167296
Tampa Bay Lightning,1571.867775
Minnesota Wild,1569.853126
Boston Bruins,1564.511377
St. Louis Blues,1563.661118
Calgary Flames,1562.634962


In [88]:
print("Playoff Probabilities")

playoff_df = pd.DataFrame(columns = ['Playoff Prob']).rename_axis('Team')

playoffs_counter_normalized = {team: count / num_simulations for team, count in playoffs_counter.items()}

for team, playoff_prob in playoffs_counter_normalized.items():
    playoff_df.loc[team] = [playoff_prob]

playoff_df.sort_values(by='Playoff Prob', ascending=False)

Playoff Probabilities


,Playoff Prob
Team,
Vegas Golden Knights,0.8926
Colorado Avalanche,0.8802
Boston Bruins,0.8329
Tampa Bay Lightning,0.8326
Pittsburgh Penguins,0.8032
Carolina Hurricanes,0.7968
Washington Capitals,0.7896
Florida Panthers,0.7460
Toronto Maple Leafs,0.7354


In [13]:
season_2022

,Date,Visitor,G_Vis,Home,G_Home,OT Ind,Season,EloDiffHome,EloDiffAway,WinProbHome,WinProbVis
23662,2021-10-12,Pittsburgh Penguins,1.0,Tampa Bay Lightning,3.0,0,2022,24.150000,-24.150000,0.534699,0.465301
23663,2021-10-12,Seattle Kraken,2.0,Vegas Golden Knights,1.0,0,2022,82.260000,-82.260000,0.616218,0.383782
23664,2021-10-13,Winnipeg Jets,7.0,Anaheim Ducks,1.0,0,2022,-36.470000,36.470000,0.447707,0.552293
23665,2021-10-13,Chicago Blackhawks,1.0,Colorado Avalanche,0.0,0,2022,100.820000,-100.820000,0.641152,0.358848
23666,2021-10-13,Vancouver Canucks,6.0,Edmonton Oilers,1.0,0,2022,72.510000,-72.510000,0.602861,0.397139
...,...,...,...,...,...,...,...,...,...,...,...
24969,2022-04-29,San Jose Sharks,2.0,Seattle Kraken,4.0,0,2022,75.722038,-75.722038,0.607280,0.392720
24970,2022-04-29,Vegas Golden Knights,1.0,St. Louis Blues,5.0,0,2022,13.081857,-13.081857,0.518817,0.481183
24971,2022-04-29,Boston Bruins,5.0,Toronto Maple Leafs,2.0,0,2022,-47.719030,47.719030,0.431755,0.568245
24972,2022-04-29,Calgary Flames,2.0,Winnipeg Jets,4.0,0,2022,116.315306,-116.315306,0.661406,0.338594


In [89]:
# 2022-23 Season
real_scores = np.array([['Boston Bruins',135],['Toronto Maple Leafs',111],['Tampa Bay Lightning',98],
              ['Florida Panthers',92],['Buffalo Sabres',91],['Ottawa Senators',86],['Detroit Red Wings',80],
              ['Montreal Canadiens',68],['Carolina Hurricanes',113],['New Jersey Devils',112],['New York Rangers',107],
              ['New York Islanders',93],['Pittsburgh Penguins',91],['Washington Capitals',80],['Philadelphia Flyers',75],
              ['Columbus Blue Jackets',59],['Colorado Avalanche',109],['Dallas Stars',108],['Minnesota Wild',103],
              ['Winnipeg Jets',95],['Nashville Predators',92],['St. Louis Blues',81],['Arizona Coyotes',70],
              ['Chicago Blackhawks',59],['Vegas Golden Knights',111],['Edmonton Oilers',109],['Los Angeles Kings',104],
              ['Seattle Kraken',100],['Calgary Flames',93],['Vancouver Canucks',83],['San Jose Sharks',60],['Anaheim Ducks',58]])

real_scores = pd.DataFrame(real_scores)
real_scores = real_scores.rename(columns={0:'Team', 1:'PTS'})
real_scores['PTS'] = pd.to_numeric(real_scores['PTS'])

In [90]:
points_df = pd.merge(mean_pts, real_scores, on='Team')
points_df['Diff'] = (np.abs(points_df['PTS_y']-points_df['PTS_x']))
display(points_df)

,Team,PTS_x,PTS_y,Diff
0,Nashville Predators,93.3676,92,1.3676
1,San Jose Sharks,78.1848,60,18.1848
2,Los Angeles Kings,79.4855,104,24.5145
3,New York Rangers,91.6652,107,15.3348
4,Anaheim Ducks,75.3090,58,17.3090
5,Carolina Hurricanes,100.3925,113,12.6075
6,Colorado Avalanche,103.3856,109,5.6144
7,Edmonton Oilers,94.9998,109,14.0002
8,Montreal Canadiens,83.3457,68,15.3457
9,Washington Capitals,100.4537,80,20.4537


In [91]:
mean_x = np.mean(points_df['PTS_x'])
std_dev_x = np.std(points_df['PTS_x'])

mean_y = np.mean(points_df['PTS_y'])
std_dev_y = np.std(points_df['PTS_y'])

mean_diff = np.mean(points_df['Diff'])
std_dev_diff = np.std(points_df['Diff'])

test = pd.DataFrame({'Mean':[mean_x, mean_y, mean_diff],'Std Dev':[std_dev_x, std_dev_y, std_dev_diff]})
test.rename({0:'Model', 1:'Actual', 2:'Diff'},axis=0)

,Mean,Std Dev
Model,89.593553,9.530655
Actual,91.437500,18.600970
Diff,12.888903,8.608875


In [92]:
MeanAE = np.sum(points_df['Diff'])/points_df['Diff'].shape[0]
RSS = np.sum((points_df['Diff'])**2)
print(f'The Mean Absolute Error is: {MeanAE}')
print(f'The Residual Sum of Squares is: {RSS}')

The Mean Absolute Error is: 12.888903124999999
The Residual Sum of Squares is: 7687.569514849998


In [93]:
playoffs_2023 = np.array([['Boston Bruins',1],['Toronto Maple Leafs',1],['Tampa Bay Lightning',1],
              ['Florida Panthers',1],['Buffalo Sabres',0],['Ottawa Senators',0],['Detroit Red Wings',0],
              ['Montreal Canadiens',0],['Carolina Hurricanes',1],['New Jersey Devils',1],['New York Rangers',1],
              ['New York Islanders',1],['Pittsburgh Penguins',0],['Washington Capitals',0],['Philadelphia Flyers',0],
              ['Columbus Blue Jackets',0],['Colorado Avalanche',1],['Dallas Stars',1],['Minnesota Wild',1],
              ['Winnipeg Jets',1],['Nashville Predators',0],['St. Louis Blues',0],['Arizona Coyotes',0],
              ['Chicago Blackhawks',0],['Vegas Golden Knights',1],['Edmonton Oilers',1],['Los Angeles Kings',1],
              ['Seattle Kraken',1],['Calgary Flames',0],['Vancouver Canucks',0],['San Jose Sharks',0],['Anaheim Ducks',0]])

playoffs_2023=pd.DataFrame(playoffs_2023)
playoffs_2023=playoffs_2023.rename(columns={0:'Team', 1:'PTS'})

real_scores_2022 = np.array([['Florida Panthers',122],['Toronto Maple Leafs',115],['Tampa Bay Lightning',110],
              ['Boston Bruins',107],['Buffalo Sabres',75],['Detroit Red Wings',74],['Ottawa Senators',73],
              ['Montreal Canadiens',55],['Carolina Hurricanes',116],['New York Rangers',110],['Pittsburgh Penguins',103],
              ['Washington Capitals',100],['New York Islanders',84],['Columbus Blue Jackets',81],['New Jersey Devils',63],
              ['Philadelphia Flyers',61],['Colorado Avalanche',119],['Minnesota Wild',113],['St. Louis Blues',109],
              ['Dallas Stars',98],['Nashville Predators',97],['Winnipeg Jets',89],['Chicago Blackhawks',68],
              ['Arizona Coyotes',57],['Calgary Flames',111],['Edmonton Oilers',104],['Los Angeles Kings',99],
              ['Vegas Golden Knights',94],['Vancouver Canucks',92],['San Jose Sharks',77],['Anaheim Ducks',76],['Seattle Kraken',60]])

playoffs_2022 = np.array([['Florida Panthers',1],['Toronto Maple Leafs',1],['Tampa Bay Lightning',1],
              ['Boston Bruins',1],['Buffalo Sabres',0],['Detroit Red Wings',0],['Ottawa Senators',0],
              ['Montreal Canadiens',0],['Carolina Hurricanes',1],['New York Rangers',1],['Pittsburgh Penguins',1],
              ['Washington Capitals',1],['New York Islanders',0],['Columbus Blue Jackets',0],['New Jersey Devils',0],
              ['Philadelphia Flyers',0],['Colorado Avalanche',1],['Minnesota Wild',1],['St. Louis Blues',1],
              ['Dallas Stars',1],['Nashville Predators',1],['Winnipeg Jets',0],['Chicago Blackhawks',0],
              ['Arizona Coyotes',0],['Calgary Flames',1],['Edmonton Oilers',1],['Los Angeles Kings',1],
              ['Vegas Golden Knights',0],['Vancouver Canucks',0],['San Jose Sharks',0],['Anaheim Ducks',0],['Seattle Kraken',0]])

playoffs_2022=pd.DataFrame(playoffs_2022)
playoffs_2022=playoffs_2022.rename(columns={0:'Team', 1:'PTS'})

playoffs_2019 = np.array([['Tampa Bay Lightning',1],['Boston Bruins',1],['Toronto Maple Leafs',1],
              ['Montreal Canadiens',0],['Florida Panthers',0],['Buffalo Sabres',0],['Detroit Red Wings',0],
              ['Ottawa Senators',0],['Washington Capitals',1],['New York Islanders',1],['Pittsburgh Penguins',1],
              ['Carolina Hurricanes',1],['Columbus Blue Jackets',1],['Philadelphia Flyers',0],['New York Rangers',0],
              ['New Jersey Devils',0],['Nashville Predators',1],['Winnipeg Jets',1],['St. Louis Blues',1],
              ['Dallas Stars',1],['Colorado Avalanche',1],['Chicago Blackhawks',0],['Minnesota Wild',0],
              ['Calgary Flames',1],['San Jose Sharks',1],['Vegas Golden Knights',1],['Arizona Coyotes',0],
              ['Vancouver Canucks',0],['Anaheim Ducks',0],['Edmonton Oilers',0],['Los Angeles Kings',0]])

playoffs_2019=pd.DataFrame(playoffs_2019)
playoffs_2019=playoffs_2019.rename(columns={0:'Team', 1:'PTS'})

In [96]:
playoff_score = pd.merge(playoff_df, playoffs_2023, on='Team')
playoff_score['PTS'] = pd.to_numeric(playoff_score['PTS'], errors='coerce')
playoff_score

,Team,Playoff Prob,PTS
0,Nashville Predators,0.6338,0
1,San Jose Sharks,0.1762,0
2,Los Angeles Kings,0.2133,1
3,New York Rangers,0.5415,1
4,Anaheim Ducks,0.1254,0
5,Carolina Hurricanes,0.7968,1
6,Colorado Avalanche,0.8802,1
7,Edmonton Oilers,0.6803,1
8,Montreal Canadiens,0.2870,0
9,Washington Capitals,0.7896,0


In [97]:
abs(playoff_score['Playoff Prob']- playoff_score['PTS']).mean()

0.36226875000000003